In [13]:
source_file = "/Users/varunpasad/projects/odoo/wastage/import/wastage.xlsx"
product_file = ""
# mrp_file = "/Users/varunpasad/projects/odoo/tcuts/data/marinades/mrp.bom.csv"
# mrp_line_file = "/Users/varunpasad/projects/odoo/tcuts/data/marinades/mrp.bom.csv"

source_file = "/Users/varunpasad/projects/odoo/wastage/import/wastage.xlsx"
mrp_file = "/Users/varunpasad/projects/odoo/wastage/import/mrp.bom.csv"
mrp_line_file = "/Users/varunpasad/projects/odoo/wastage/import/mrp.bom.line.csv"

In [14]:
# load the sheet
import pandas as pd
import numpy as np

class MarinadeCatalogGen:
    ING_NAME = "ingredients"
    RECEIPE_NAME = "receipe"

    def __init__(self):
        #self.sheet_name = "/Users/varunpasad/Downloads/export_marinades - FILLED.xlsx"
        self.sheet_name = source_file
        self.ing_df = None
        self.int_df = None
        self.receipe_df = None
    
    def fetch_products(self):
        sheet_db = pd.read_excel(self.sheet_name, sheetname=[
            self.ING_NAME,
            self.RECEIPE_NAME
        ])
       
        self.ing_df = sheet_db[self.ING_NAME]
        self.receipe_df = sheet_db[self.RECEIPE_NAME]

        
class ProductTemplGen:
    # GEnerate catalog
    SKU_TO_CAT = {
        "CHK_": "tcuts.product_category_chicken",
        "GT_": "tcuts.product_category_goat",
        "LAMB_": "tcuts.product_category_goat",
        "SF_": "tcuts.product_category_seafood",
        "MRT_":  "tcuts.product_category_marinades",

        "RM_CHK_": "tcuts.product_category_chicken",
        "RM_GT_": "tcuts.product_category_goat",
        "RM_MUT_": "tcuts.product_category_goat",
        "RM_LAMB_": "tcuts.product_category_goat",
        "RM_SF_" : "tcuts.product_category_seafood",
        "RM_MRT_": "tcuts.product_category_marinades",
        
        # MRT
        "MRTI_" : "tcuts.product_category_marinades_inter",
        "ING_" : "tcuts.product_category_ingredients"
    }

    def __init__(self, df):
        self.df = df
    
    def reformat_sheet(self):
        """Format the sheet to odoo understandable cols"""
        self.df = self.df.reset_index()[['sku', 'name', 'cost', 'expiry']]
        self.df = self.df.rename(columns={
            'sku': 'default_code',
            'cost':'standard_price'})
        self.df = self.df.set_index('default_code')
    
    def add_default_cols(self):
        """Adds necessary cols
        
        type, tracking, active (based on dep) and finally generated XID
        """
        # external id
        self.df['type']  = "Stockable Product"
        self.df['tracking']  = "lot"
        self.df = self.df.reset_index().apply(self.build_row, axis=1)

        # convert _DEP to inactive in odoo and build the ID
        self.df['active'] = np.where(
            self.df.default_code.str.endswith("_DEP"), "FALSE", "TRUE")
        self.df.default_code = self.df.default_code.str.replace(
            r"^(.*)_DEP$", r"\1")

        self.df = self.df.set_index("default_code")
        self.df['id'] = "tcuts.product_" + self.df.index
        # drop off anything that does not have category
        self.df = self.df[~self.df['categ_id/id'].isnull()]

        # set active/inactive
    
    def is_raw_material(self, sku):
        if sku.startswith("RM_"):
            return True
        
        if sku.startswith("MRTI_"):
            return True
        
        if sku.startswith("ING_"):
            return True

        return False
    
    def build_row(self, row):
        """Generate the row data
        such as salelable, pruchasable etc
        """
        sku = row['default_code']
        is_raw = self.is_raw_material(sku)

        for part, cat_name in self.SKU_TO_CAT.items():
            if sku.startswith(part):
                row['categ_id/id'] = cat_name
                break

        # SKU
        if not is_raw:
            row['purchase_ok'] = "FALSE"
            row['sale_ok'] = "TRUE"
            row['use_time'] = 2
            row['alert_time'] = 1
            row['removal_time'] = 2
            row['life_time'] = 2
            row['uom_po_id/id'] = "product.product_uom_kgm"
            row['uom_id/id'] = "product.product_uom_kgm"
        else:
            row['purchase_ok'] = "TRUE"
            row['sale_ok'] = "TRUE"
            row['use_time'] = 2
            row['alert_time'] = 2
            row['removal_time'] = 3
            row['life_time'] = 4
            row['uom_po_id/id'] = "product.product_uom_kgm"
            row['uom_id/id'] = "product.product_uom_kgm"

        # no taxes
        row['taxes_id'] = None
        row['supplier_taxes_id'] = None

        return row

    def generate_catalog(self):
        self.reformat_sheet()
        self.add_default_cols()
    
    def to_csv(self, path):
        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=[
                "id", "default_code", "name", "type", "tracking", "categ_id/id",
                "uom_po_id/id", "uom_id/id", "purchase_ok", "sale_ok",
                "use_time", "alert_time", "removal_time", "life_time",
                "taxes_id", "supplier_taxes_id", "standard_price", "active"]
            )
 
class BomGen:
    def __init__(self, df, variant=True):
        """sku, parent_sku,weight"""
        self.df = df
        self.variant = variant
    
    def reformat_cols(self):
        self.df = self.df.reset_index()[['sku', 'weight', 'variant']]

        self.df = self.df.rename(columns={
            "weight": "product_qty"
        })
        
        # add active flag.
        self.df['active'] = np.where(
            self.df.sku.str.endswith("_DEP"), "FALSE", "TRUE")
        
        # And remove the _DEP from the name.
        self.df.sku = self.df.sku.str.replace(r"^(.*)_DEP$", r"\1")
        
        # generate product ID:
        self.df['product_tmpl_id/id'] = "tcuts.product_" + self.df.sku

        # generate Ids
        self.df['id'] = "tcuts.mrp_bom_" + self.df.sku
        if self.variant:
            self.df.id = self.df.id + "_" + self.df.variant.astype(int).astype(str)
            del self.df['variant']
        
        # inject addl cols
        self.df['type'] = "Manufacture this product"
        self.df['product_uom_id/id'] = 'product.product_uom_kgm'
    
    
    def generate_bom(self):
        # rolling fill
        self.df = self.df.fillna(method='ffill')
        self.reformat_cols()
        self.df = self.df.drop_duplicates('id')
        
        # now on to bom.line
    
    def to_csv(self, path):
        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=["id", "product_tmpl_id/id", "product_qty", "type", "active", "product_uom_id/id"]
        )
        
class BomLineGen:
    def __init__(self, df):
        """sku,variant,weight,name,parent_sku,qty"""
        self.df = df
    
    def reformat_cols(self):
        
        # remove dep
        self.df.sku = self.df.sku.str.replace(r"^(.*)_DEP$", r"\1")
        self.df.parent_sku = self.df.parent_sku.str.replace(r"^(.*)_DEP$", r"\1")
        
        # generat the BOM id of the SKU
        self.df['bom_id/id'] = "tcuts.mrp_bom_" + self.df.sku
        self.df['bom_id/id'] = \
            self.df['bom_id/id'] + "_" + self.df.variant.astype(int).astype(str)
        
        # generate product ID for the BOM(parent_Sku)
        self.df['product_id/id'] = "tcuts.product_" + self.df.parent_sku
    
        # finally the bom line ID:
        self.df['id'] = \
            "tcuts.bom_line_" + self.df.sku + "_" + self.df.variant.astype(int).astype(str) \
            + "_" + self.df.recipe_index.astype(int).astype(str)
        
        # add cons cols
        self.df['product_uom_id/id'] = 'product.product_uom_kgm'
        self.df['product_qty'] = self.df.qty
        
        self.df = self.df[['id', 'product_id/id', 'product_qty', 'product_uom_id/id', 'bom_id/id']]

    def generate_bom(self):
        # rolling fill
        self.df = self.df.fillna(method='ffill')
        self.reformat_cols()
        
    def to_csv(self, path):
        self.df.reset_index().to_csv(
            path,
            index=False,
            columns=['id', 'product_id/id', 'product_qty', 'product_uom_id/id', 'bom_id/id']
        )
    
        


In [15]:
# build the base sheet
gen = MarinadeCatalogGen()
gen.fetch_products()

prod = ProductTemplGen(gen.ing_df)
#prod.generate_catalog()
# prod.to_csv("/Users/varunpasad/projects/odoo/tcuts/data/marinades/product.template.csv")

bomgen = BomGen(gen.receipe_df)
bomgen.generate_bom()
bomgen.to_csv(mrp_file)

bomlinegen = BomLineGen(gen.receipe_df)
bomlinegen.generate_bom()
bomlinegen.to_csv(mrp_line_file)
